In [1]:
import plotly.express as px
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
%matplotlib inline

sns.set_style('darkgrid')
matplotlib.rcParams['font.size'] = 14
matplotlib.rcParams['figure.figsize'] = (10, 6)
matplotlib.rcParams['figure.facecolor'] = '#00000000'
import pandas as pd 
dfDeliveries = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\IPL_Ball_by_Ball_2008_2022.csv") 
dfMatches = pd.read_csv(r"C:\Users\hp\Desktop\MAJOR\data\archive\IPL_Matches_2008_2022.csv")
dfDeliveries = dfDeliveries.merge(dfMatches, on='ID')
dfDeliveries['BowlingTeam'] = np.where(dfDeliveries['BattingTeam']==dfDeliveries['Team1'],
                                       dfDeliveries['Team2'],
                                       dfDeliveries['Team1'])



dfDeliveries['delivery'] = 1
dfDeliveries['Dot_Balls'] = (dfDeliveries['batsman_run'] == 0).astype(int)
dfDeliveries['Singles'] = (dfDeliveries['batsman_run'] == 1).astype(int)
dfDeliveries['Fours'] = (dfDeliveries['batsman_run'] == 4).astype(int)
dfDeliveries['Sixes'] = (dfDeliveries['batsman_run'] == 6).astype(int)


def overClassifier(val):
    if val>=0 and val<6:
        return 'Power Play'
    elif val>=6 and val<15:
        return 'Middle Overs'
    else:
        return 'Death Overs'
dfDeliveries['PhaseOfPlay'] = dfDeliveries['overs'].map(overClassifier)
dfDeliveries[['overs','PhaseOfPlay']].sort_values('overs').drop_duplicates().reset_index(drop=True)
dfDeathOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Death Overs']
dfPowerplayOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Power Play Overs']
dfMiddleOvers = dfDeliveries[dfDeliveries['PhaseOfPlay']=='Middle Overs']

df=dfDeliveries

new_df = df.head(1000).copy()



In [8]:
import pandas as pd

cricket_data =dfDeliveries


# Convert relevant columns to numeric data types
numeric_columns = ['innings', 'overs', 'batsman_run', 'total_run']
cricket_data[numeric_columns] = cricket_data[numeric_columns].apply(pd.to_numeric, errors='coerce')

# Create a function to calculate batting impact
def calculate_batting_impact(df):
    # Assuming you have columns like 'innings', 'overs', 'batsman_run', 'total_run', 'Venue', 'WinningTeam', etc.
    
    # Create a new column for Batting Impact
    df['BattingImpact'] = 0.0
    
    # Define factors and weights
    weights = {
        'innings': 0.2,
        'overs': 0.1,
        'batsman_run': 0.5,
        'total_run': 0.3,
        'Venue': 0.2,
        # Exclude 'WinningTeam' from consideration
    }

    # Calculate Batting Impact for each row
    for index, row in df.iterrows():
        # Skip rows where the team didn't win
        if row['WinningTeam'] == 'No Result' or row['WinningTeam'] == 'Tie' or pd.isna(row['WinningTeam']):
            continue

        batting_impact = 0.0
        
        for factor, weight in weights.items():
            value = row[factor]
            
            # Check if both the value and weight are numeric
            if pd.notna(value) and pd.notna(weight) and isinstance(value, (int, float)) and isinstance(weight, (int, float)):
                batting_impact += value * weight
        
        df.at[index, 'BattingImpact'] = batting_impact
    
    return df

# Apply the function to calculate Batting Impact
cricket_data = calculate_batting_impact(cricket_data)

# Display the DataFrame with Batting Impact
print(cricket_data[['batter', 'BattingImpact']])






             batter  BattingImpact
0       YBK Jaiswal            0.2
1       YBK Jaiswal            0.5
2        JC Buttler            1.0
3       YBK Jaiswal            0.2
4       YBK Jaiswal            0.2
...             ...            ...
225949      P Kumar            2.1
225950     SB Joshi            2.6
225951      P Kumar            1.8
225952     SB Joshi            2.2
225953     SB Joshi            1.9

[225954 rows x 2 columns]


In [18]:
new_df.columns


Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'batsman_run', 'extras_run', 'total_run',
       'isWicketDelivery', 'player_out', 'Venue', 'WinningTeam', 'delivery',
       'Dot_Balls', 'Singles', 'Fours', 'Sixes', 'PhaseOfPlay'],
      dtype='object')

In [35]:
df=new_df

# Assuming your DataFrame is named 'df'
# Replace 'YourDataFrame' with the actual name of your DataFrame if it's different

# Sort the DataFrame by 'ID', 'innings', 'overs', and 'ballnumber'
df = df.sort_values(by=['ID', 'innings', 'overs', 'ballnumber'])

# Create a new column to store the cumulative runs
df['CumulativeRuns'] = df.groupby(['ID', 'innings'])['total_run'].cumsum() - df['total_run']

# Fill NaN values with 0 for the first ball in each match
df['CumulativeRuns'] = df['CumulativeRuns'].fillna(0)
df['CumulativeWickets'] = df.groupby(['ID', 'innings'])['isWicketDelivery'].cumsum() - df['isWicketDelivery']
df['CumulativeBalls'] = df.groupby(['ID', 'innings'])['ballnumber'].cumcount()

# Fill NaN values with 0 for the first ball in each match
df['CumulativeWickets'] = df['CumulativeWickets'].fillna(0)
df['CumulativeBalls'] = df['CumulativeBalls'].fillna(0)

df['PrevBowler'] = df.groupby(['ID', 'innings'])['bowler'].shift(1)
df['PrevBowler'] = df['PrevBowler'].fillna('')

df['Prevbatter'] = df.groupby(['ID', 'innings'])['batter'].shift(1)
df['Prevbatter'] = df['Prevbatter'].fillna('')

df['PrevRuns'] = df.groupby(['ID', 'innings'])['total_run'].shift(1)
df['PrevRuns'] = df['PrevRuns'].fillna(0)

df['Prevwickets'] = df.groupby(['ID', 'innings'])['isWicketDelivery'].shift(1)
df['Prevwickets'] = df['Prevwickets'].fillna(0)

# Reset the DataFrame index to its default
df = df.reset_index(drop=True)

# Display the updated DataFrame
df[['ID', 'innings', 'overs', 'ballnumber','batter', 'bowler',
 'total_run','CumulativeRuns','CumulativeWickets','CumulativeBalls','PrevBowler','Prevbatter','PrevRuns','Prevwickets']].tail(25)

,ID,innings,overs,ballnumber,batter,bowler,total_run,CumulativeRuns,CumulativeWickets,CumulativeBalls,PrevBowler,Prevbatter,PrevRuns,Prevwickets
975,1312200,2,14,2,Shubman Gill,OC McCoy,1,90,3,88,OC McCoy,DA Miller,1.0,0.0
976,1312200,2,14,3,DA Miller,OC McCoy,0,91,3,89,OC McCoy,Shubman Gill,1.0,0.0
977,1312200,2,14,4,DA Miller,OC McCoy,4,91,3,90,OC McCoy,DA Miller,0.0,0.0
978,1312200,2,14,5,DA Miller,OC McCoy,1,95,3,91,OC McCoy,DA Miller,4.0,0.0
979,1312200,2,14,6,Shubman Gill,OC McCoy,1,96,3,92,OC McCoy,DA Miller,1.0,0.0
980,1312200,2,15,1,Shubman Gill,R Ashwin,2,97,3,93,OC McCoy,Shubman Gill,1.0,0.0
981,1312200,2,15,2,Shubman Gill,R Ashwin,1,99,3,94,R Ashwin,Shubman Gill,2.0,0.0
982,1312200,2,15,3,DA Miller,R Ashwin,6,100,3,95,R Ashwin,Shubman Gill,1.0,0.0
983,1312200,2,15,4,DA Miller,R Ashwin,1,106,3,96,R Ashwin,DA Miller,6.0,0.0
984,1312200,2,15,5,Shubman Gill,R Ashwin,1,107,3,97,R Ashwin,DA Miller,1.0,0.0


In [39]:
df=new_df
#COLUMN DROP GARNUPARLA 
df = df.sort_values(by=['ID', 'innings', 'overs', 'ballnumber'])

df['CumulativeRuns'] = df.groupby(['ID', 'innings'])['total_run'].cumsum() - df['total_run']
df['CumulativeRuns'] = df['CumulativeRuns'].fillna(0)

df['CumulativeWickets'] = df.groupby(['ID', 'innings'])['isWicketDelivery'].cumsum() - df['isWicketDelivery']
df['CumulativeBalls'] = df.groupby(['ID', 'innings'])['ballnumber'].cumcount()
df['CumulativeWickets'] = df['CumulativeWickets'].fillna(0)
df['CumulativeBalls'] = df['CumulativeBalls'].fillna(0)

for i in range(1, 7):
    df[f'PrevBowler_{i}'] = df.groupby(['ID', 'innings'])['bowler'].shift(i).fillna('')
    df[f'PrevBatter_{i}'] = df.groupby(['ID', 'innings'])['batter'].shift(i).fillna('')
    df[f'PrevRuns_{i}'] = df.groupby(['ID', 'innings'])['total_run'].shift(i).fillna(0)
    df[f'PrevWickets_{i}'] = df.groupby(['ID', 'innings'])['isWicketDelivery'].shift(i).fillna(0)
df.columns

Index(['ID', 'innings', 'overs', 'ballnumber', 'batter', 'bowler',
       'non-striker', 'batsman_run', 'extras_run', 'total_run',
       'isWicketDelivery', 'player_out', 'Venue', 'WinningTeam', 'delivery',
       'Dot_Balls', 'Singles', 'Fours', 'Sixes', 'PhaseOfPlay', 'PrevBowler_1',
       'PrevBatter_1', 'PrevRuns_1', 'PrevWickets_1', 'PrevBowler_2',
       'PrevBatter_2', 'PrevRuns_2', 'PrevWickets_2', 'PrevBowler_3',
       'PrevBatter_3', 'PrevRuns_3', 'PrevWickets_3', 'PrevBowler_4',
       'PrevBatter_4', 'PrevRuns_4', 'PrevWickets_4', 'PrevBowler_5',
       'PrevBatter_5', 'PrevRuns_5', 'PrevWickets_5', 'PrevBowler_6',
       'PrevBatter_6', 'PrevRuns_6', 'PrevWickets_6', 'CumulativeRuns',
       'CumulativeWickets', 'CumulativeBalls'],
      dtype='object')